In [6]:
from read_data import process_batch

In [2]:
user = "m"
if user == "v":
    sample_data_dir = "/Users/virenbajaj/Desktop/Columbia Fall 20/Graphical Models/project/20200705v1/sample/"
    full_data_dir = "/Users/virenbajaj/Desktop/Columbia Fall 20/Graphical Models/project/HighResLDA/SemanticScholar Data/20200705v1/full/"
else:
    full_data_dir = './Dataset/20200705v1/full/'
    
metadata_dir = full_data_dir + 'metadata/'
pdf_parses_dir = full_data_dir + 'pdf_parses/'

In [3]:
files = range(1)
metadata = [metadata_dir + f'metadata_{i}.jsonl.gz' for i in files]
pdfs = [pdf_parses_dir + f'pdf_parses_{i}.jsonl.gz' for i in files]
fields = ['Computer Science', 'Medicine']
#get_ids = set(flat_links) # from cell that gets links of all refs
print(metadata, pdfs)

['./Dataset/20200705v1/full/metadata/metadata_0.jsonl.gz'] ['./Dataset/20200705v1/full/pdf_parses/pdf_parses_0.jsonl.gz']


In [7]:
for batch in zip(metadata,pdfs):
    process_batch(batch[0], batch[1])

100%|████████████████████████████████████████████████████████████████████████| 310736/310736 [1:43:53<00:00, 49.85it/s]
